# Large Loss Threshold Implementation
The Jupyter Notebook aims to implement a reserving methodology which focuses on valuing attritional claims and large claims separately. In theory, the removal of large losses will result in a more stable attritional triangle which should yield better predictions of future claims development. 

Large losses will be defined as claims above a threshold determined by the largest change in Mack Standard Error relative to the removal of the largest claims in the data. Claim size will be defined as the following:
- For incurred triangles, claim size of an individual claim is calculated as the cumulative value of paid claims and the maximum OCR amount held at any point
- For paid triangles, claim size of an individual claim is calculated as the cumulative value of paid claims

Large claims above the threshold should be assessed separately and a separate "large" IBNR should be raised to support these claims.

The triangle is incurred and thus will focus on an incurred approach. Where this approach is applied to paid data instead, the use of OCR in the threshold process should be removed.

## 0. Setting up the Environment

### 0.1. Use a virtual environment (Optional)
A virtual environment is a useful method of ensuring that any packages installed for code are specfic to the current project. We will use `venv` for this project and a list of packages can then be installed using the `requirements.txt` file. This will reduce the need of individually installing each package and will ensure that the environment is appropriate for running the code. Use the following steps:

1. Install the virtualenv package using the command below
2. Open the terminal and run the command `virtualenv venv` which will create a folder called venv with the required information
3. Run the command `.\venv\Scripts\activate` in the terminal which will active the virtual environment
4. Packages can then be installed through pip individually (like the virtualenv package below) or through the `requirement.txt` file (demonstrated 2 cells below)
5. The virtualenv package can stopped at any time by running the command "deactivate" in the terminal

In [ ]:
%pip install virtualenv

If using the virtual environment, follow steps 2 and 3 above before proceeding. Instructions are based on powershell.

In [ ]:
%pip install -r .\requirement.txt

### 0.2. Import packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import chainladder as cl
from dateutil.relativedelta import relativedelta
from datetime import date, timedelta
import os
from tqdm import tqdm
import warnings
import sqlite3

warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

### 0.3. Important variables

In [2]:
data_path = ""

conn = sqlite3.connect(':memory:')

## 1. Determining Large Loss Threshold

Outline of methodology:
- Organise claims data
- Remove x% of largest claims
- Calculate mack error
- Iteratively remove x% of claims and calculate mack error
- Plot the mack error and % of claims removed (rebase from 0 to 1 for both)
- Select the threshold which results in the steepest gradient

### 1.1. Extracting and organising data

In [ ]:
# Extracting the Gross Earned Premium Data
gep_data = pd.read_excel(data_path, sheet_name="RW Gross EP").drop(columns=["Unnamed: 0", "Unnamed: 12"])
gep_data = pd.melt(gep_data, id_vars=["Accident Period"], var_name="Segment", value_name="GEP")
gep_data['Segment'] = gep_data['Segment'].str.title()
gep_col = gep_data.columns
gep_data = gep_data.rename(columns={
    gep_col[0]:"loss_date",
    gep_col[1]:"segment",
    gep_col[2]:"gep"
})

gep_data.head()

In [ ]:
# Extracting the Net Earned Premium Data
nep_data = pd.read_excel(data_path, sheet_name="RW Net EP").drop(columns=["Unnamed: 0", "Unnamed: 12"])
nep_data = pd.melt(nep_data, id_vars=["Accident Period"], var_name="Segment", value_name="NEP")
nep_data['Segment'] = nep_data['Segment'].str.title()
nep_col = nep_data.columns
nep_data = nep_data.rename(columns={
    nep_col[0]:"loss_date",
    nep_col[1]:"segment",
    nep_col[2]:"nep"
})

nep_data.head()

In [ ]:
# Combining the Gross and Net Earned Premium Data
combined_ep = gep_data.merge(nep_data, on=["loss_date", "segment"])
assert gep_data.shape == nep_data.shape
assert gep_data.shape[0] == combined_ep.shape[0]

combined_ep.head()

In [ ]:
# Extracting the Gross and Net Paid Claims Data
paid_data = pd.read_excel(data_path, sheet_name="RW Gross_Net Paid 122023")
paid_col = paid_data.columns
paid_data = paid_data.rename(columns={
    paid_col[0]:"claim_id",
    paid_col[1]:"segment",
    paid_col[2]:"loss_date", 
    paid_col[3]:"transaction_date",
    paid_col[4]:"gross_paid",
    paid_col[5]:"net_paid"
})
paid_data['segment'] = paid_data['segment'].str.title()

paid_data.head()

In [ ]:
# Extracting the Gross and Net Outstanding Claims Data

ocr_data = pd.read_excel(data_path, sheet_name="RW Gross_Net RBNS 122023")

ocr_col = ocr_data.columns
ocr_data = ocr_data.rename(columns={
    ocr_col[0]:"claim_id",
    ocr_col[1]:"segment",
    ocr_col[2]:"loss_date", 
    ocr_col[3]:"transaction_date",
    ocr_col[4]:"gross_ocr",
    ocr_col[5]:"net_ocr"
})
ocr_data["loss_date"] = pd.to_datetime(ocr_data["loss_date"])
ocr_data['segment'] = ocr_data['segment'].str.title()

ocr_data.head()

Noted when looking at the data that there were 5 claims in which the loss date changed for one of the transactions, however, this occured once towards the later periods. This issue will be corrected by using the loss date associated with the earliest transaction date.

In [ ]:
ocr_data_sorted = ocr_data.sort_values(by=['claim_id', 'transaction_date'])

# Drop duplicates to keep the first occurrence for each 'claim_id'
ocr_data_sorted = ocr_data_sorted.drop_duplicates(subset=['claim_id'], keep='first')[["claim_id", "loss_date"]]
ocr_data_sorted.head()

In [10]:
ocr_data = ocr_data.drop(columns=["loss_date"]).merge(ocr_data_sorted, how="left", on=["claim_id"])

In [ ]:
ocr_data.to_sql('ocr_claims', conn, index=False, if_exists='replace')
paid_data.to_sql('paid_claims', conn, index=False, if_exists='replace')
combined_ep.to_sql('earned_premiums', conn, index=False, if_exists='replace')

In [ ]:
print(
    pd.read_sql_query(
    """
        SELECT 
        segment, 
        SUM(CASE WHEN loss_date IS NULL THEN gross_ocr ELSE 0 END) AS ocr_amount_blank_loss_date,
        SUM(CASE WHEN loss_date IS NOT NULL THEN gross_ocr ELSE 0 END) AS ocr_amount_non_blank_loss_date,
        SUM(CASE WHEN loss_date IS NULL THEN gross_ocr ELSE 0 END) * 1.0 / SUM(gross_ocr) AS proportion_blank_to_all
        FROM ocr_claims
        GROUP BY segment;
    """, conn)
)

# Negligible proportion of blank loss date claims, blank loss data claims only noted in Engineering segment

### 1.1. Organising Claims Data

The Rwanda valuation data provided consists of paid and OCR data. In assessing the data, the following has been noted:
- Paid data is incremental
- OCR data is cumulative

This was assessed by recreating the triangles for the December valuations with no differences noted. Instances where a loss date was blank resulted in the data being removed from the triangle

In [ ]:
ocr_claims = pd.read_sql_query(
    """
        SELECT 
        claim_id,
        segment,
        loss_date,
        MAX(gross_ocr) as gross_amount,
        MAX(net_ocr) as net_amount
        FROM ocr_claims
        GROUP BY claim_id, segment, loss_date;
    """, conn).dropna(subset=["loss_date"])

ocr_claims.head()

In [ ]:
paid_claims = pd.read_sql_query(
    """
        SELECT 
        claim_id,
        segment,
        loss_date,
        SUM(gross_paid) as gross_amount,
        SUM(net_paid) as net_amount
        FROM paid_claims
        GROUP BY claim_id, segment, loss_date;
    """, conn).dropna(subset=["loss_date"])

paid_claims.head()

In [ ]:
combined_df = (
    pd.concat([ocr_claims, paid_claims])
)

combined_df = combined_df.groupby(['claim_id','segment', 'loss_date']).agg(
    gross_amount=pd.NamedAgg(column="gross_amount", aggfunc="sum"),
    net_amount=pd.NamedAgg(column="net_amount", aggfunc="sum")
    ).reset_index()
combined_df.head()

### 1.2. Identifying Largest Claims 

In [ ]:
segment_list = ocr_claims["segment"].unique()
plot_path = "./plots"
threshold_percentage = {}

if not os.path.exists(plot_path):
    os.makedirs(plot_path)

for segment in segment_list[:4]:
    claims_percentage = []
    mack_mse = []
    threshold = []

    start_point = 1
    end_point = 0.7
    increment = 0.01
    interval = int(round((start_point-end_point)/increment,0))

    segment_df = combined_df.loc[combined_df["segment"]==segment]
    segment_paid = paid_data.loc[paid_data["segment"]==segment]
    segment_ocr = ocr_data.loc[ocr_data["segment"]==segment]
    
    print(f"{segment}: {interval}")

    for i in range(interval+1):
        attritional_claims = segment_df.sort_values(by='gross_amount', ascending=False)
        one_percent_index = int(len(attritional_claims) * increment*i)
        attritional_claims = attritional_claims.iloc[one_percent_index:]

        if one_percent_index>0:
            segment_paid = segment_paid.merge(attritional_claims["claim_id"], on="claim_id")
            segment_ocr = segment_ocr.merge(attritional_claims["claim_id"], on="claim_id")

        paid_triangle = cl.Triangle(
            data=segment_paid,
            origin="loss_date",
            development="transaction_date",
            columns=["gross_paid"],
            cumulative=False
        ).grain("OQDQ").incr_to_cum()

        ocr_triangle = cl.Triangle(
            data=segment_ocr,
            origin="loss_date",
            development="transaction_date",
            columns=["gross_ocr"],
            cumulative=True
        ).grain("OQDQ")

        combined_triangle = paid_triangle + ocr_triangle
        
        mack = cl.MackChainladder()
        dev = cl.Development(average='volume')
        mack.fit(dev.fit_transform(combined_triangle))
        plot_data = mack.summary_.to_frame(origin_as_datetime=False)
        # print(plot_data["Mack Std Err"].sum())
        
        claims_percentage.append(start_point-increment*i)
        mack_mse.append(plot_data[["Mack Std Err"]].sum()[0])
    
    segment_dict = {"claims_percentage": claims_percentage, "mack_mse": mack_mse}
        
    mack_df = pd.DataFrame(segment_dict)

    # Calculating the largest drop in mack_mse
    mack_df['mack_mse_diff'] = mack_df['mack_mse'].diff().abs()
    max_drop_index = mack_df['mack_mse_diff'][1:].idxmax()
    # percentile_with_max_drop = mack_df.loc[max_drop_index, 'claims_percentage']
    largest_drop = mack_df.loc[max_drop_index, 'mack_mse_diff']
    threshold_percentage[f"Rwanda {segment}"] = mack_df.loc[max_drop_index, 'claims_percentage']
    print(attritional_claims)
    
    plt.figure(figsize=(10, 6))
    plt.scatter(mack_df['claims_percentage'], mack_df['mack_mse'], label='Data Points')
    plt.scatter(mack_df.loc[max_drop_index, 'claims_percentage'], mack_df.loc[max_drop_index, 'mack_mse'], color='red', label='Largest Drop')
    plt.axvline(x=mack_df.loc[max_drop_index, 'claims_percentage'], color='red', linestyle='--', label='Drop Indicator')
    plt.xlabel('Claims Percentage')
    plt.ylabel('Mack SE')
    plt.title(f"Rwanda {segment}: Mack SE")
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(plot_path, f"Rwanda_{segment}_mack_se"))
    plt.show()    

    large_claims = segment_df.sort_values(by='gross_amount', ascending=False).iloc[:one_percent_index]

    print(large_claims)
# 010/021/9/000391/2021

In [ ]:
claims_percentage = []
mack_mse = []

start_point = 1
end_point = 0.9
increment = 0.0025
interval = int(round((start_point-end_point)/increment,0))

for i in range(interval+1):
    print(cleaned_df)
    filtered_df = cleaned_df.sort_values(by='amount', ascending=False)
    one_percent_index = int(len(filtered_df) * increment*i)
    filtered_df = filtered_df.iloc[one_percent_index:]
    
    triangle = cl.Triangle(
        data=filtered_df,
        origin="accident_date",
        development="transaction_date",
        columns=["amount"],
        index="lob",
        cumulative=False
    ).grain("OQDQ")
    
    mack = cl.MackChainladder()
    mack.full_triangle_
    dev = cl.Development(average='volume')
    mack.fit(dev.fit_transform(triangle))

    plot_data = mack.summary_.to_frame(origin_as_datetime=False)
    
    claims_percentage.append(start_point-increment*i)
    mack_mse.append(plot_data[["Mack Std Err"]].sum()[0])
    
iterated_mack = {"claims_percentage": claims_percentage, "mack_mse": mack_mse}
iterated_mack

In [ ]:
ordered_df = combined_df.sort_values(by='gross_amount', ascending=False)
one_percent_index = int(len(ordered_df) * 0.005)
ordered_df = ordered_df.iloc[one_percent_index:]
ordered_df.head()

In [ ]:
cleaned_df = sample_df.drop(columns=['FIN_MONTH', "large_attritional"])
cleaned_df = cleaned_df.rename(columns={"Class": "lob", "Loss Date": "accident_date", "Pay Date": "transaction_date", cleaned_df.columns[3]: "amount"})
cleaned_df["lob"] = cleaned_df["lob"].str.title()

cleaned_df['accident_date'] = pd.to_datetime(cleaned_df['accident_date'], errors='coerce', infer_datetime_format=False).dt.strftime('%d/%m/%Y')
cleaned_df['accident_date'] = pd.to_datetime(cleaned_df['accident_date'], format='%d/%m/%Y', errors='coerce')
cleaned_df['transaction_date'] = pd.to_datetime(cleaned_df['transaction_date'], errors='coerce').dt.strftime('%d/%m/%Y')
cleaned_df['transaction_date'] = pd.to_datetime(cleaned_df['transaction_date'], format='%d/%m/%Y', errors='coerce')
# cleaned_df = cleaned_df.loc[cleaned_df['accident_date'] >= pd.to_datetime('01/01/2006')]
# cleaned_df = cleaned_df.loc[cleaned_df['transaction_date'] >= pd.to_datetime('01/01/2006')]

cleaned_df.head()

In [ ]:
claims_percentage = []
mack_mse = []

start_point = 1
end_point = 0.9
increment = 0.0025
interval = int(round((start_point-end_point)/increment,0))

for i in range(interval+1):
    print(cleaned_df)
    filtered_df = cleaned_df.sort_values(by='amount', ascending=False)
    one_percent_index = int(len(filtered_df) * increment*i)
    filtered_df = filtered_df.iloc[one_percent_index:]
    
    triangle = cl.Triangle(
        data=filtered_df,
        origin="accident_date",
        development="transaction_date",
        columns=["amount"],
        index="lob",
        cumulative=False
    ).grain("OQDQ")
    
    mack = cl.MackChainladder()
    mack.full_triangle_
    dev = cl.Development(average='volume')
    mack.fit(dev.fit_transform(triangle))

    plot_data = mack.summary_.to_frame(origin_as_datetime=False)
    
    claims_percentage.append(start_point-increment*i)
    mack_mse.append(plot_data[["Mack Std Err"]].sum()[0])
    
iterated_mack = {"claims_percentage": claims_percentage, "mack_mse": mack_mse}
iterated_mack

In [ ]:
print(iterated_mack)

In [ ]:
sample_df = sample_df.sort_values(by='amount', ascending=False)
one_percent_index = int(len(sample_df) * 0.005)
sample_df = sample_df.iloc[one_percent_index:]
sample_df

In [ ]:
triangle = cl.Triangle(
        data=sample_df,
        origin="accident_date",
        development="transaction_date",
        columns=["amount"],
        index="lob",
        cumulative=False
    ).grain("OQDQ")

triangle

In [ ]:
mack = cl.MackChainladder()
dev = cl.Development(average='volume')
mack.fit(dev.fit_transform(triangle))

plot_data = mack.summary_.to_frame(origin_as_datetime=False)
plot_data[["Mack Std Err"]].sum()[0]